In [1]:
# Dependencies
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import requests
import numpy as np
import pandas as pd

In [2]:
# Setup splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 92.0.4515
Get LATEST driver version for 92.0.4515
Get LATEST driver version for 92.0.4515
Trying to download new driver from https://chromedriver.storage.googleapis.com/92.0.4515.107/chromedriver_win32.zip
Driver has been saved in cache [C:\Users\helena\.wdm\drivers\chromedriver\win32\92.0.4515.107]


In [3]:
# URL of page to be scraped
# url = 'http://house.speakingsame.com/p.php?q=Perth%2C+WA'
url = 'http://house.speakingsame.com/p.php?q=West+Leederville%2C+WA'
browser.visit(url)

In [4]:
addresses = []
properties_info = []

pages = np.arange(0, 30, 1)

for page in pages:
    
#     page_url = "http://house.speakingsame.com/p.php?q=Perth&p="+ str(page) +"&s=1&st=&type=&count=300&region=Perth&lat=0&lng=0&sta=wa&htype=&agent=0&minprice=0&maxprice=0&minbed=0&maxbed=0&minland=0&maxland=0"
    page_url = "http://house.speakingsame.com/p.php?q=West+Leederville&p="+ str(page) +"&s=1&st=&type=&count=300&region=West+Leederville&lat=0&lng=0&sta=wa&htype=&agent=0&minprice=0&maxprice=0&minbed=0&maxbed=0&minland=0&maxland=0"
    
    browser.visit(page_url)
   
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')

    addr_spans = soup.find_all('span', class_='addr')

    for span in addr_spans:
        addresses.append(span.a.text)
    addresses

    tables = soup.find_all('table', style="font-size:13px") #tables list (except 0,1 index) contians list of properties in the page

    for i in range(2,12):
        try:
            info = []
            trs = tables[i].find_all('tr')
            for tr in trs:
                info.append(tr.td.text)
        except:
            info.append('')
        properties_info.append(info)
        
  
    
properties_info  

[['Sold $680,000 in Aug 2021',
  'Apartment: ',
  'Land size: 143 sqm',
  'Agent: Craig Gaspar',
  'Street view | Nearby'],
 ['Sold $237,000 in Jul 2021',
  'Last Sold $215,000 in May 2012',
  'Apartment: 1  1  1 ',
  'Land size: 4,059 sqm',
  'Agent: Emma Taylor 0421331468',
  'Street view | Nearby'],
 ['Sold $1,020,000 in Jul 2021',
  'Rent $900pw in Jun 2014',
  'Apartment: 3  2  1 ',
  'Land size: 646 sqm | Building size: 121 sqm',
  'Agent: Ivan Helman',
  "The Ultimate Inner-City Penthouse Living with sky garden!. Looking for something really special? Uniquely positioned with easy access to to the city and Perth's Western Suburbs, unit 8 at 3...more",
  'Street view | Nearby'],
 ['Sold $750,000 in Jul 2021', 'Land: ', 'Agent: Craig Gaspar', ''],
 ['Sold $945,000 in Jul 2021',
  'House: ',
  'Land size: 359 sqm',
  'Lot/Plan No: P001564 32more',
  'Street view | Nearby'],
 ['Sold $1,060,000 in Jul 2021',
  'Last Sold $900,000 in Nov 2012',
  'House: 4  2  1 ',
  'Land size: 355 sq

In [5]:
len(properties_info)

300

In [6]:
len(addresses)

300

In [7]:
Properties = []
for i in range(0, len(addresses)):
    
    Property_dict = {}
    Property_dict['address'] = addresses[i]
    
    for j in range (0, len(properties_info[i])):
        
        try:
                    
            if properties_info[i][j].split(' ')[0] == "Sold": 
                Property_dict['price'] = properties_info[i][j].split(' ')[1]
                Property_dict['sold date'] = properties_info[i][j].split('in ')[1]

            if properties_info[i][j].split(' ')[0] in ["Apartment:", "House:", "Townhouse:"]:
                Property_dict['type'] = properties_info[i][j].split(' ')[0].split(':')[0]
                Property_dict['bedrooms'] = properties_info[i][j].split(' ')[1]
                try:
                    Property_dict['bathrooms'] = properties_info[i][j].split(' ')[3]
                except:
                    Property_dict['bathrooms'] = ''
                try:
                    Property_dict['car space'] = properties_info[i][j].split(' ')[5]
                except:
                    Property_dict['car space'] = ''
                    
            if properties_info[i][j].split(' ')[0] == "Agent:":
                Property_dict['agent'] = properties_info[i][j].split(': ')[1]
                
            if properties_info[i][j].split(' ')[0] == "Rent":
                Property_dict['rent'] = properties_info[i][j].split(' ')[1]
                Property_dict['rent date'] = properties_info[i][j].split('in ')[1]
                
            if properties_info[i][j].split(' ')[1] == "size:":
                if properties_info[i][j].split(' ')[0] == "Land":
                    Property_dict['Land size'] = properties_info[i][j].split(' ')[2]
                    try:
                        if properties_info[i][j].split(' ')[5] == "Building":
                            Property_dict['Building size'] = properties_info[i][j].split(' ')[7]
                    except:
                        continue
                elif properties_info[i][j].split(' ')[0] == "Building":
                    Property_dict['Building size'] = properties_info[i][j].split(' ')[2]
            
            
        except:
            continue

  
    Properties.append(Property_dict)
        
Properties

[{'address': '9/21 Northwood Street',
  'price': '$680,000',
  'sold date': 'Aug 2021',
  'type': 'Apartment',
  'bedrooms': '',
  'bathrooms': '',
  'car space': '',
  'Land size': '143',
  'agent': 'Craig Gaspar'},
 {'address': '4/32 Cambridge Street',
  'price': '$237,000',
  'sold date': 'Jul 2021',
  'type': 'Apartment',
  'bedrooms': '1',
  'bathrooms': '1',
  'car space': '1',
  'Land size': '4,059',
  'agent': 'Emma Taylor 0421331468'},
 {'address': '8/3 Rosslyn Street',
  'price': '$1,020,000',
  'sold date': 'Jul 2021',
  'rent': '$900pw',
  'rent date': 'Jun 2014',
  'type': 'Apartment',
  'bedrooms': '3',
  'bathrooms': '2',
  'car space': '1',
  'Land size': '646',
  'Building size': '121',
  'agent': 'Ivan Helman'},
 {'address': '71A Tate Street',
  'price': '$750,000',
  'sold date': 'Jul 2021',
  'agent': 'Craig Gaspar'},
 {'address': '19 Caddy Avenue',
  'price': '$945,000',
  'sold date': 'Jul 2021',
  'type': 'House',
  'bedrooms': '',
  'bathrooms': '',
  'car space

In [8]:
len(Properties)

300

In [9]:
WestLeederville_df = pd.DataFrame(Properties)

In [10]:
WestLeederville_df

,address,price,sold date,type,bedrooms,bathrooms,car space,Land size,agent,rent,rent date,Building size
0,9/21 Northwood Street,"$680,000",Aug 2021,Apartment,,,,143,Craig Gaspar,NaN,NaN,NaN
1,4/32 Cambridge Street,"$237,000",Jul 2021,Apartment,1,1,1,"4,059",Emma Taylor 0421331468,NaN,NaN,NaN
2,8/3 Rosslyn Street,"$1,020,000",Jul 2021,Apartment,3,2,1,646,Ivan Helman,$900pw,Jun 2014,121
3,71A Tate Street,"$750,000",Jul 2021,NaN,NaN,NaN,NaN,NaN,Craig Gaspar,NaN,NaN,NaN
4,19 Caddy Avenue,"$945,000",Jul 2021,House,,,,359,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
295,8 St Leonards Avenue,"$1,725,000",Mar 2017,House,4,2,1,536,Growth Realty - Subiaco,NaN,NaN,282
296,14/110 Cambridge Street,"$580,000",Mar 2017,Apartment,2,2,1,79,M&M Property Consultant - Leederville,$460pw,Jun 2020,79
297,23 Brighton Street,"$740,000",Mar 2017,House,3,1,2,304,Acton - Dalkeith,$525pw,Nov 2015,NaN
298,102 Kimberley Street,"$1,125,000",Mar 2017,House,3,2,1,291,Caporn Young Estate Agents - Claremont,$750pw,Sep 2019,175


In [11]:
WestLeederville_df.to_csv("WestLeederville_properties.csv")